In [ ]:
import leo_satellites as sat
import csv
import ephem
import geometry_functions as geom
import math
from collections import defaultdict
import sys
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import sys
import math

In [ ]:
def constellationFromSaVi(tcl_observed_date, tcl_file_name, satellite_per_orbit, number_of_obits, flat=False):
    constellation = list()
    with open(tcl_file_name, 'r') as tclfile:
        spaceX_SaVi = list(csv.reader(tclfile, delimiter='\n'))
        if len(spaceX_SaVi) != satellite_per_orbit * number_of_obits:
            raise Exception('The total number of satellites is not correct')

    for i in range(number_of_obits):
        orbit_i = list()
        for j in range(satellite_per_orbit):
            SaVi_line = spaceX_SaVi.pop(0)[0].split()
            for i in range (0, 2):
                SaVi_line.pop(0)
            to_add_sat = ephem.EarthSatellite()
            # TBD：历元是否影响结果-
            to_add_sat._epoch = tcl_observed_date #ok
            to_add_sat._n = geom.semi_major_to_mean_motion(float(SaVi_line[0])) #ok
            to_add_sat._e = float(SaVi_line[1]) + math.pow(10, -30) #ok
            to_add_sat._inc = float(SaVi_line[2]) #ok
            to_add_sat._raan = float(SaVi_line[3]) #ok
            to_add_sat._ap = float(SaVi_line[4]) #ok
            to_add_sat._M = geom.time_to_periapsis_to_mean_anomaly(float(SaVi_line[5]),to_add_sat._n) #ok
            to_add_sat._drag = 0
            orbit_i.append(to_add_sat)
        if flat is True:
            constellation.extend(orbit_i)
        else:
            constellation.append(orbit_i)
    return constellation


def get_observer(observe_time, lat, lon):
    observer = ephem.Observer()
    observer.lat = ephem.degrees(lat)
    observer.lon = ephem.degrees(lon)
    observer.date = observe_time
    return observer

def get_observer_list(observe_time, lat_start, lat_end, lon_start, lon_end, lat_n, lon_n):
    observer_list = []
    
    for i in range(lat_n):
        for j in range(lon_n):
            lat = (lat_end - lat_start) / lat_n * i + lat_start
            lon = (lon_end - lon_start) / lon_n * j + lon_start
            observer = ephem.Observer()
            observer.lat = ephem.degrees(lat)
            observer.lon = ephem.degrees(lon)
            observer.date = observe_time
            observer_list.append(observer)
    return observer_list

In [ ]:
def get_max_accessible_seconds_since(accessible_time_set, t):
    if t not in accessible_time_set:
        return 0
    max_t = t + 1
    max_accessible_seconds = 0
    while True:
        if max_t in accessible_time_set:
            max_accessible_seconds += 1
        else:
            return max_accessible_seconds
        max_t += 1

In [ ]:
def get_max_accessible_seconds(
    t, max_channel_per_satellite, satellite_idx, satellite_list, observer_idx, observer_list, accessible_time, satellite_link_count):
    return get_max_accessible_seconds_since(accessible_time[observer_idx][satellite_idx], t)

def get_rss(t, max_channel_per_satellite, satellite_idx, satellite_list, observer_idx, observer_list, accessible_time, satellite_link_count):
    satellite = satellite_list[satellite_idx]
    observer = observer_list[observer_idx]
    satellite.compute(observer)
    distance_in_meter = satellite.range
    rss = -43 - 40 * math.log(distance_in_meter / 1000 / 1000, 10)
    return rss

def get_free_channel(t, max_channel_per_satellite, satellite_idx, satellite_list, observer_idx, observer_list, accessible_time, satellite_link_count):
    return max_channel_per_satellite - satellite_link_count[satellite_idx]

In [ ]:
SATELLITES_PER_ORBIT = 22
NUMBER_OF_ORBITS = 72
TCL_FILE_NAME = './spacex_constellation/spacex_phase1_550km.tcl'
# Shanghai
SHANGHAI_LAT_START = ephem.degrees('30:40:00')
SHANGHAI_LON_START = ephem.degrees('120:51:00')

LONDON_LAT = ephem.degrees('51:30:26')
LONDON_LON = ephem.degrees('-0:8:11')
TCL_OBSERVED_DATE = ephem.date('2022/09/21 00:00:00')


observe_start_time = ephem.date('2022/09/21 00:00:00')
observe_total_seconds = 16000
satellite_list = constellationFromSaVi(TCL_OBSERVED_DATE, TCL_FILE_NAME, SATELLITES_PER_ORBIT, NUMBER_OF_ORBITS, flat=True)
lat = SHANGHAI_LAT_START
lon = SHANGHAI_LON_START
min_service_degrees=ephem.degrees('40')

In [ ]:
intervals = []

for satellite_idx, satellite in enumerate(satellite_list):
    t = 0
    # try to find out possible reachable time range
    while t < observe_total_seconds:
        observe_time = ephem.date(observe_start_time + t / 24 / 60 / 60)
        observer = get_observer(observe_time, lat, lon)
        satellite.compute(observer)
        if satellite.alt >= min_service_degrees:
            # we find a possible interval
            rss_list = []
            distance_km_list = []
            start_t = t
            while t < observe_total_seconds:
                observe_time = ephem.date(observe_start_time + t / 24 / 60 / 60)
                observer = get_observer(observe_time, lat, lon)
                satellite.compute(observer)
                if satellite.alt >= min_service_degrees:
                    distance_in_meter = satellite.range
                    rss = -43 - 40 * math.log(distance_in_meter / 1000 / 1000, 10)
                    distance_km_list.append(distance_in_meter / 1000)
                    rss_list.append(rss)
                    t += 1
                else:
                    break
            visible_time = t - start_t
            rss_sum = sum(rss_list)
            # for convenience
            # at time t, actually there's no signal
            # at start_t, there's signal
            intervals.append({
                'satellite_idx': satellite_idx,
                'start': start_t,
                'end': t,
                'visible_time': visible_time,
                'rss_sum': rss_sum,
                'mean_rss': rss_sum / (t - start_t),
                'mean_distance_km': sum(distance_km_list) / len(distance_km_list)
            })
        else:
            t += 1

In [ ]:
intervals = sorted(intervals, key=lambda r: r['start'])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=[interval['mean_rss'] for interval in intervals],
    y=[interval['visible_time'] for interval in intervals],
    mode='markers')
)

fig.update_layout(
    title="visible_time v.s. mean_rss",
    width=900,
    height=400,
)
fig.show()


In [ ]:
import plotly.figure_factory as ff
import numpy as np
np.random.seed(1)

hist_data = [np.array([interval['mean_rss'] for interval in intervals])]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, bin_size=0.2)
fig.show()

In [ ]:
hist_data = [np.array([interval['mean_distance_km'] for interval in intervals])]
group_labels = ['distplot'] # name of the dataset
hist_data = [np.array([interval['mean_distance_km'] for interval in intervals])]
fig = ff.create_distplot(hist_data, group_labels, bin_size=10)
fig.show()

In [ ]:
mean_distance_km_array = np.array([interval['mean_distance_km'] for interval in intervals])

normalized_distance_list = (mean_distance_km_array - np.min(mean_distance_km_array)) / (np.max(mean_distance_km_array) - np.min(mean_distance_km_array))
normalized_distance_list = normalized_distance_list.tolist()
for i, interval in enumerate(intervals):
    interval['normalized_distance'] = normalized_distance_list[i]

### problem formulation

Given:

- simulation_time: 0 to T seconds
- N intervals: interval_i = (start_i, end_i, value_i), while ** value_i should be larger than 0**
- choose intervals from all intervals and minimize the sum of value_i, and it should cover 0..T
- if v_i = 1, we're just optimizing the handover count

DP algorithm

- f[t]: stands for the ans from 0 to the t second.

In [ ]:
# represent tick 0, tick 1, ..., tick observe_total_seconds - 1
# interval stands for [tick start, tick end)
f = [100000000000000000] * observe_total_seconds
f[0] = 0
chosen_interval = [None] * observe_total_seconds
prev_f = [None] * observe_total_seconds
prev_f[0] = 0
# alpha_list = [i for i in range(0, 10, 1)]
alpha_list = [0, 1, 2, 3]
handover_count_list = []
mean_rss_list = []

print(alpha_list)
for alpha in alpha_list:
    print(alpha)
    for t in range(1, observe_total_seconds):
        for interval_idx, interval in enumerate(intervals):
            # for f[t], tick t is not necessarily covered by the interval, hence t <= interval['end']
            # if t == interval['end'], e.g. f[2], {'start': 1, 'end': 2}, we allow it.
            if interval['start'] < t <= interval['end']:
#                 value = pow(alpha, interval['normalized_distance'] / 0.1)
                value = pow(interval['normalized_distance'], alpha)
                if f[t] > f[interval['start']] + value:
                    f[t] = f[interval['start']] + value
                    chosen_interval[t] = interval_idx
                    prev_f[t] = interval['start']

    selected_interval_idx = []

    trace_t = observe_total_seconds - 1
    while prev_f[trace_t] != trace_t:
        selected_interval_idx.append(chosen_interval[trace_t])
        trace_t = prev_f[trace_t]
    selected_interval_idx = selected_interval_idx[::-1]
    for idx in selected_interval_idx:
        interval = intervals[idx]
    selected_satellite_idx = []
    p = 0
    selected_satellite_idx.append(intervals[selected_interval_idx[p]]['satellite_idx'])
    for t in range(1, observe_total_seconds):
        interval = intervals[selected_interval_idx[p]]
        if interval['start'] < t and interval['end'] >= t:
            selected_satellite_idx.append(interval['satellite_idx'])
        else:
            p += 1
            interval = intervals[selected_interval_idx[p]]
            assert interval['start'] < t and interval['end'] >= t
            selected_satellite_idx.append(interval['satellite_idx'])

    rss_list = []
    for t in range(observe_total_seconds):
        satellite = satellite_list[selected_satellite_idx[t]]
        observe_time = ephem.date(observe_start_time + t / 24 / 60 / 60)
        observer = get_observer(observe_time, lat, lon)
        satellite.compute(observer)
        distance_in_meter = satellite.range
        rss = -43 - 40 * math.log(distance_in_meter / 1000 / 1000, 10)
        rss_list.append(rss)
    mean_rss = sum(rss_list) / len(rss_list)
    handover_count = len(selected_interval_idx)
    mean_rss_list.append(mean_rss)
    handover_count_list.append(handover_count)

In [ ]:
print(mean_rss_list)
print(handover_count_list)

In [ ]:
for idx, interval in enumerate(intervals):
    print('idx: {:<8}  start: {:<8} end:  {:<8} visible_time: {:<8} value : {:<8}'.format(
        idx, interval['start'], interval['end'], interval['visible_time'], '{:.2f}'.format(pow(5, interval['normalized_distance'] / 0.1))))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=alpha_list,
    y=mean_rss_list,
    mode='markers+lines+text',
    text=['{:.2f}'.format(mean_rss) for mean_rss in mean_rss_list],
    textposition="bottom center",
    name='london')
)

fig.update_layout(
    title="mean rss v.s. alpha",
    width=500,
    height=400,
)
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=alpha_list,
    y=handover_count_list,
    mode='markers+lines+text',
    text=['{}'.format(cnt) for cnt in handover_count_list],
    textposition="bottom center",
    name='london')
)

fig.update_layout(
    title="handover count v.s. alpha",
    width=500,
    height=400,
)
fig.show()


In [ ]:
def do_simulate(
    observe_start_time,
    observe_total_seconds,
    satellite_list,
    lat_start,
    lat_end,
    lon_start,
    lon_end,
    lat_n,
    lon_n,
    min_service_degrees,
    max_link_per_satellite,
    no_signal_rss=-100,
    name_and_evaluate_func_info=[("by_mvt", get_max_accessible_seconds)],
    ):
    accessible_time = defaultdict(lambda: defaultdict(set))
    for t in range(observe_total_seconds):
        observe_time = ephem.date(observe_start_time + t / 24 / 60 / 60)
        observer_list = get_observer_list(observe_time, lat_start, lat_end, lon_start, lon_end, lat_n, lon_n)
        for i, observer in enumerate(observer_list):
            for j, satellite in enumerate(satellite_list):
                satellite.compute(observer)
                if satellite.alt >= min_service_degrees:
                    accessible_time[i][j].add(t)

    observer_rss_log = defaultdict(lambda: defaultdict(list))
    observer_handover_detail_log = defaultdict(lambda: defaultdict(list))
    reachable_satellite_count_log = defaultdict(lambda: defaultdict(list))

    for name, evaluate_func in name_and_evaluate_func_info:
        satellite_link_count = defaultdict(int)
        observer2linked_statellite = [None] * len(observer_list)

        for t in range(observe_total_seconds):
            observe_time = ephem.date(observe_start_time + t / 24 / 60 / 60)
            observer_list = get_observer_list(observe_time, lat_start, lat_end, lon_start, lon_end, lat_n, lon_n)
            for i, observer in enumerate(observer_list):
                previous_satellite_idx = observer2linked_statellite[i]
                candidate_list = []
                for satellite_idx in accessible_time[i]:
                    if t in accessible_time[i][satellite_idx]:
                        if satellite_link_count[satellite_idx] < max_link_per_satellite:
                            candidate_list.append(satellite_idx)
                reachable_satellite_count_log[name][i].append(len(candidate_list))
                # if the observer is not connected, or the previous connected satellite is out of service
                if previous_satellite_idx is None or t not in accessible_time[i][previous_satellite_idx]:
                    # try to find an available satellite
                    if len(candidate_list) > 0:
                        # we can find a candidate
                        best_satellite_idx = max(candidate_list, key=lambda satellite_idx: evaluate_func(
                            t, 
                            max_link_per_satellite,
                            satellite_idx,
                            satellite_list,
                            i,
                            observer_list,
                            accessible_time,
                            satellite_link_count
                        ))
                        if previous_satellite_idx is not None:
                            satellite_link_count[previous_satellite_idx] -= 1
                        observer2linked_statellite[i] = best_satellite_idx
                        satellite_link_count[best_satellite_idx] += 1
                        satellite = satellite_list[best_satellite_idx]
                        satellite.compute(observer)
                        distance_in_meter = satellite.range
                        rss = -43 - 40 * math.log(distance_in_meter / 1000 / 1000, 10)
                        observer_rss_log[name][i].append(rss)
                        observer_handover_detail_log[name][i].append(1)
                    else:
                        # we cannot find a candidate, no handover happens, but there's no signal at all
                        observer_rss_log[name][i].append(no_signal_rss)
                        observer_handover_detail_log[name][i].append(0)
                else:
                    # observer is ok, we records its signal 
                    linked_satellite_idx = observer2linked_statellite[i]
                    satellite = satellite_list[linked_satellite_idx]
                    satellite.compute(observer)
                    distance_in_meter = satellite.range
                    rss = -43 - 40 * math.log(distance_in_meter / 1000 / 1000, 10)
                    observer_rss_log[name][i].append(rss)
                    observer_handover_detail_log[name][i].append(0)
    return {
        'observer_rss_log': observer_rss_log,
        'observer_handover_detail_log': observer_handover_detail_log,
        'reachable_satellite_count_log': reachable_satellite_count_log,
    }

In [ ]:
SATELLITES_PER_ORBIT = 22
NUMBER_OF_ORBITS = 72
TCL_FILE_NAME = './spacex_constellation/spacex_phase1_550km.tcl'

# Shanghai
LAT_START = ephem.degrees('30:40:00')
LAT_END = ephem.degrees('31:53:00')
LON_START = ephem.degrees('120:51:00')
LON_END = ephem.degrees('122:12:00')

MIN_SERVICE_DEGREES = ephem.degrees('40')

TCL_OBSERVED_DATE = ephem.date('2022/09/21 00:00:00')
MAX_LINK_PER_SATELLITE = 10

NO_SIGNAL_RSS = -100

## don't consider max_link_per_satellite

### reachable count

In [ ]:
satellite_list = constellationFromSaVi(TCL_OBSERVED_DATE, TCL_FILE_NAME, SATELLITES_PER_ORBIT, NUMBER_OF_ORBITS, flat=True)

# Shanghai
SHANGHAI_LAT_START = ephem.degrees('30:40:00')
SHANGHAI_LAT_END = ephem.degrees('31:53:00')
SHANGHAI_LON_START = ephem.degrees('120:51:00')
SHANGHAI_LON_END = ephem.degrees('122:12:00')

ret_shanghai = do_simulate(
    ephem.date('2022/09/21 00:00:00'),
    1600,
    satellite_list,
    SHANGHAI_LAT_START,
    SHANGHAI_LAT_END,
    SHANGHAI_LON_START,
    SHANGHAI_LON_END,
    1,
    1,
    min_service_degrees=ephem.degrees('40'),
    max_link_per_satellite=10,
    name_and_evaluate_func_info=[
        ("by_mvt", get_max_accessible_seconds),
    ]
)

# Hongkong
HONGKONG_LAT_START = ephem.degrees('22:08:00')
HONGKONG_LAT_END = ephem.degrees('22:35:00')
HONGKONG_LON_START = ephem.degrees('113:49:00')
HONGKONG_LON_END = ephem.degrees('114:31:00')

ret_hongkong = do_simulate(
    ephem.date('2022/09/21 00:00:00'),
    1600,
    satellite_list,
    HONGKONG_LAT_START,
    HONGKONG_LAT_END,
    HONGKONG_LON_START,
    HONGKONG_LON_END,
    1,
    1,
    min_service_degrees=ephem.degrees('40'),
    max_link_per_satellite=10,
    name_and_evaluate_func_info=[
        ("by_mvt", get_max_accessible_seconds),
    ]
)

# London
LONDON_LAT_START = ephem.degrees('51:30:26')
LONDON_LAT_END = ephem.degrees('51:30:26')
LONDON_LON_START = ephem.degrees('-0:8:11')
LONDON_LON_END = ephem.degrees('-0:8:11')

ret_london = do_simulate(
    ephem.date('2022/09/21 00:00:00'),
    1600,
    satellite_list,
    LONDON_LAT_START,
    LONDON_LAT_END,
    LONDON_LON_START,
    LONDON_LON_END,
    1,
    1,
    min_service_degrees=ephem.degrees('40'),
    max_link_per_satellite=10,
    name_and_evaluate_func_info=[
        ("by_mvt", get_max_accessible_seconds),
    ]
)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=ret_shanghai['reachable_satellite_count_log']['by_mvt'][0],
                    mode='lines',
                    name='shanghai'))

fig.add_trace(go.Scatter(y=ret_london['reachable_satellite_count_log']['by_mvt'][0],
                    mode='lines',
                    name='london'))
 
fig.add_trace(go.Scatter(y=ret_hongkong['reachable_satellite_count_log']['by_mvt'][0],
                    mode='lines',
                    name='hongkong'))
 
fig.update_layout(
    title="reachable satellite count v.s. time",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()

